In [1]:
from keras.models import Sequential,load_model
from keras.layers import Dense
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=6))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

In [2]:
import time #import time module to make some delay
import os
import cv2 #import cv2 module to capture frame from camera
from cvzone.HandTrackingModule import HandDetector # hand detector module to detect hand
import numpy as np # import numpy to make numerical calculations
import math # import module to make mathematecial calculations
import datetime # to get system date and time
cap=cv2.VideoCapture(0) #making a capture instance to activate webcam
cap.set(3,1280) # setting video window width 1280
cap.set(4,780) # setting video window height 780
detector=HandDetector(detectionCon=0.4,maxHands=1) #initialize hand detector module
model.load_weights('path_to_model_checkpoint')
class_=['victory1','victory2','other'] # defining three classes
timer=0 #initiate the capture remaining time
capture_active=False
match_counter=0 #used for making sure prediction
if not os.path.exists("Captured/"):
    os.mkdir("Captured") #if Captured folder not exist it makes the folder 
def calculate_distance(landmark_list,a,b): #FUNCTION TO CLACULATE DISTANCE BETWEEN TWO POINT OF FINGER
    landmark1,landmark2=landmark_list[a],landmark_list[b]
    p1=(landmark1[0],landmark1[1])
    p2=(landmark2[0],landmark2[1])
    dist=int(math.dist(p1,p2))
    return dist
while True:
    sttime=time.time() #start time to calculate fps
    success,frame=cap.read() #reading live frame
    if success: #if we get frame from camera
        frame=cv2.flip(frame,1) #fliping horizontaly to avoid mirror image
        frame_orig=frame.copy() #take a copy of original for saving
        hands,frame=detector.findHands(frame,flipType=False,draw=False) #detect hand
        if hands:
            lmlist=hands[0]['lmList'] #get hand points
            X_train=[] #create the input features
            x1=calculate_distance(lmlist,8,12)
            x2=calculate_distance(lmlist,4,8)
            x3=calculate_distance(lmlist,0,5)
            x4=calculate_distance(lmlist,0,17)
            x5=calculate_distance(lmlist,0,4)
            x6=calculate_distance(lmlist,3,6)
            X_train.append([x1,x2,x3,x4,x5,x6]) #append all input feaures in list
            x_test=np.array(X_train) #format with numpy array to predict
            pred=np.argmax(model.predict(x_test)[0]) #predict the index who have maximum confidence
            final=class_[pred]  # predict the final class
            if (final=='victory1' or final=='victory2') and capture_active==False:
                match_counter+=1
                if match_counter>10: #check if match stays for some time to avoid false trigger
                    capture_active=True #set capture true to ensure capture mode started
            else:
                match_counter=0
        if capture_active==True:
            timer+=1
            t=3-int(timer/30) #countdown the time to capture
            cv2.putText(frame,f"{t}",(600,350),cv2.FONT_HERSHEY_COMPLEX_SMALL,6,(0,0,255),3,cv2.LINE_AA) #show countdown
        if timer>10:
            timer=0 #make 0 to reset coundown
            capture_active=False #make capture mode false indicating capture done
            filename=datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S") #get date and time and amke a filename
            cv2.putText(frame,f"Captured",(500,350),cv2.FONT_HERSHEY_COMPLEX_SMALL,3,(0,0,255),3,cv2.LINE_AA) #showing captured
            cv2.imwrite(f"Captured/{filename}.jpg",frame_orig)  #save the image in Captured folder
            cv2.imshow("image",frame) #show the live frame
            time.sleep(2) #sleep 2 second to show capture done

        endtime=time.time() #end time to calculate fps
        fps=int(1/(endtime-sttime)) #fps calculation
        cv2.putText(frame,f"FPS:{fps}",(0,30),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2,cv2.LINE_AA) #show fps
        cv2.imshow("image",frame) #show frame in window
        k=cv2.waitKey(1) #wait for 1ms 
        if k==27:    # Esc key to stop
            break
    else:
        print("No camera found")
cv2.destroyAllWindows() #to destroy all cv2 window

1/1 [==============================] - 0s 22ms/step
